In [ ]:
"""Téléchargement du fichier zippé"""
import gdown
# https://drive.google.com/file/d/1fh9OYTWev_pYYiLdBWfY0ozrQRrR0Qce/view?usp=sharing
# https://drive.google.com/file/d/1ipklpn5o9qyTIWID_tPCtoV1uEk36ZaK/view?usp=drive_link
id_copied_on_google_drive = "1ipklpn5o9qyTIWID_tPCtoV1uEk36ZaK"
url = f"https://drive.google.com/uc?id={id_copied_on_google_drive}"

out = "Abdomen_Only_Dataset.zip"
gdown.download(url, out, quiet=False)

!unzip Abdomen_Only_Dataset.zip

Downloading...
From (original): https://drive.google.com/uc?id=1ipklpn5o9qyTIWID_tPCtoV1uEk36ZaK
From (redirected): https://drive.google.com/uc?id=1ipklpn5o9qyTIWID_tPCtoV1uEk36ZaK&confirm=t&uuid=38b35bd3-b5c8-4c65-aade-a9afb6d27cf5
To: /content/Abdomen_Only_Dataset.zip
100%|██████████| 15.2M/15.2M [00:00<00:00, 45.4MB/s]


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: Abdomen_Only_Dataset/Dataset8_Abdomen/test/original_images/01119.jpg  
  inflating: __MACOSX/Abdomen_Only_Dataset/Dataset8_Abdomen/test/original_images/._01119.jpg  
  inflating: Abdomen_Only_Dataset/Dataset8_Abdomen/test/original_images/0422.jpg  
  inflating: __MACOSX/Abdomen_Only_Dataset/Dataset8_Abdomen/test/original_images/._0422.jpg  
  inflating: Abdomen_Only_Dataset/Dataset8_Abdomen/test/original_images/01079.jpg  
  inflating: __MACOSX/Abdomen_Only_Dataset/Dataset8_Abdomen/test/original_images/._01079.jpg  
  inflating: Abdomen_Only_Dataset/Dataset8_Abdomen/test/original_images/0230.jpg  
  inflating: __MACOSX/Abdomen_Only_Dataset/Dataset8_Abdomen/test/original_images/._0230.jpg  
  inflating: Abdomen_Only_Dataset/Dataset8_Abdomen/test/original_images/0026.jpg  
  inflating: __MACOSX/Abdomen_Only_Dataset/Dataset8_Abdomen/test/original_images/._0026.jpg  
  inflating: Abdomen_Only_Dataset

In [ ]:
import glob

images = glob.glob("Abdomen_Only_Dataset/Dataset8_Abdomen/train/original_images/*.jpg")
masks = glob.glob("Abdomen_Only_Dataset/Dataset8_Abdomen/train/skin_masks/*.png")


# Conversion de l'image en LAB et réduction des couleurs
def convert_and_reduce(img, num_bins=32):
    lab_image = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    scale_factor = 256 // num_bins
    reduced_ab = (lab_image[:, :, 1:3] // scale_factor).astype(np.uint8)
    return reduced_ab


def generate_histograms(images, masks, num_bins=32):
    hist_skin = np.zeros((num_bins, num_bins), dtype=np.float32)
    hist_non_skin = np.zeros((num_bins, num_bins), dtype=np.float32)
    scale_factor = 256 // num_bins

    for img, mask in zip(images, masks):
        lab_img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        a_b = lab_img[:, :, 1:3] // scale_factor
        for x in range(img.shape[0]):
            for y in range(img.shape[1]):
                a, b = a_b[x, y]
                if mask[x, y] > 0:  # Pixel "peau"
                    hist_skin[a, b] += 1
                else:  # Pixel "non-peau"
                    hist_non_skin[a, b] += 1

    # Normaliser les histogrammes
    hist_skin /= hist_skin.sum()
    hist_non_skin /= hist_non_skin.sum()
    return hist_skin, hist_non_skin



def detect_skin(image, hist_skin, hist_non_skin, prior_skin=0.6, threshold=0.0):
    num_bins = hist_skin.shape[0]
    scale_factor = 256 // num_bins
    lab_img = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    a_b = (lab_img[:, :, 1:3] // scale_factor).astype(np.uint8)

    detected_skin = np.zeros(image.shape[:2], dtype=np.uint8)
    for x in range(image.shape[0]):
        for y in range(image.shape[1]):
            a, b = a_b[x, y]
            # Handle potential index errors
            if 0 <= a < num_bins and 0 <= b < num_bins:
                p_skin = hist_skin[a, b] * prior_skin
                p_non_skin = hist_non_skin[a, b] * (1 - prior_skin)
                # Use a threshold to improve accuracy
                if p_skin > p_non_skin and p_skin > threshold :
                    detected_skin[x, y] = 255
    return detected_skin


In [ ]:

import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# Chemin vers le dossier contenant les images et les masques
# image_dir = "Abdomen_Only_Dataset/Dataset8_Abdomen/train/original_images"
# mask_dir = "Abdomen_Only_Dataset/Dataset8_Abdomen/train/skin_masks"
import glob

images = glob.glob("Abdomen_Only_Dataset/Dataset8_Abdomen/train/original_images/*.jpg")
masks = glob.glob("Abdomen_Only_Dataset/Dataset8_Abdomen/train/skin_masks/*.png")

# Boucle pour parcourir les images et les masques
for image_path, mask_path in zip(images, masks):
  # Charger l'image et le masque
  img = cv2.imread(image_path)
  mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

  # Vérifier si l'image et le masque ont été chargés correctement
  if img is not None and mask is not None:
    # Conversion et réduction de couleurs
    reduced_ab = convert_and_reduce(img)


    # Générer les histogrammes
    hist_skin, hist_non_skin = generate_histograms([img], [mask])
    # Afficher l'image
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 3, 1)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title("Image")

    # Afficher l'histogramme
    plt.subplot(1, 3, 2)
    plt.imshow(hist_skin, cmap='viridis', extent=[0, 256, 0, 256])
    plt.title('Histogramme de la peau')
    plt.xlabel("Composante a")
    plt.ylabel("Composante b")
    plt.colorbar()
    # Détection des pixels peau
    detected_mask = detect_skin(img, hist_skin, hist_non_skin)
    plt.subplot(1, 3, 3)
    plt.imshow(detected_mask, cmap='gray')
    plt.title("Peau Détectée")
    plt.show()
    # plt.show()


    # Afficher l'image originale et le masque détecté
    # plt.figure(figsize=(10, 5))

    # plt.subplot(1, 2, 2)
    # plt.title("Peau Détectée")
    # plt.imshow(detected_mask, cmap='gray')
    # plt.show()



Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import gdown
# https://drive.google.com/file/d/1fh9OYTWev_pYYiLdBWfY0ozrQRrR0Qce/view?usp=sharing
# https://drive.google.com/file/d/1ipklpn5o9qyTIWID_tPCtoV1uEk36ZaK/view?usp=drive_link
# https://drive.google.com/file/d/16kVo3WWUvI2dqQLipV-38JqiQt5Our8-/view?usp=drive_link
id_copied_on_google_drive = "16kVo3WWUvI2dqQLipV-38JqiQt5Our8-"
url = f"https://drive.google.com/uc?id={id_copied_on_google_drive}"

out = "image_test.zip"
gdown.download(url, out, quiet=False)

!unzip image_test.zip

Downloading...
From: https://drive.google.com/uc?id=16kVo3WWUvI2dqQLipV-38JqiQt5Our8-
To: /content/image_test.zip
100%|██████████| 110k/110k [00:00<00:00, 3.17MB/s]

Archive:  image_test.zip
   creating: imag_test/
  inflating: imag_test/images (1).jpg  
  inflating: imag_test/images (2).jpg  
  inflating: imag_test/images (3).jpg  
  inflating: imag_test/images (4).jpg  
  inflating: imag_test/images (5).jpg  
  inflating: imag_test/images (6).jpg  
  inflating: imag_test/images.jpg    
  inflating: imag_test/istockphoto-504481783-612x612.jpg  
  inflating: imag_test/téléchargement (1).jpg  
  inflating: imag_test/téléchargement (2).jpg  
  inflating: imag_test/téléchargement (3).jpg  
  inflating: imag_test/téléchargement (4).jpg  
  inflating: imag_test/téléchargement (5).jpg  
  inflating: imag_test/téléchargement (6).jpg  
  inflating: imag_test/téléchargement.jpg  


In [ ]:
import glob

imgs = glob.glob("/content/imag_test/*.jpg")
# print(type(imgs))
# masks = glob.glob("Abdomen_Only_Dataset/Dataset8_Abdomen/train/skin_masks/*.png")
for img in imgs:
  # Charger l'image et le masque
  img = cv2.imread(img)

  # Afficher l'image
  plt.figure(figsize=(10, 5))
  plt.subplot(1, 2, 1)
  plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
  plt.title("Image")

  detected_mask = detect_skin(img, hist_skin, hist_non_skin)
  plt.subplot(1, 2, 2)
  plt.imshow(detected_mask, cmap='gray')
  plt.title("Peau Détectée")
  plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')